In [12]:
import gym
from torch.nn import functional as F
import torch
import torch.nn as nn

In [13]:
class Agent(nn.Module):
    def __init__(self):
        super(Agent,self).__init__()
        
        self.dense1= nn.Linear(4,15)
        self.dense2= nn.Linear(15,15)
        
        self.dense3= nn.Linear(15,2)
        
        
    def forward(self,x):
        
        out = self.dense1(x)
        out = F.relu(out)
        
        out = F.dropout(out)
        
        out = self.dense2(out)
        out = F.relu(out)
        
        out = self.dense3(out)
        out = F.softmax(out,dim=-1)
        
        
        return out

In [14]:
net = Agent()

In [15]:
net

Agent(
  (dense1): Linear(in_features=4, out_features=15, bias=True)
  (dense2): Linear(in_features=15, out_features=15, bias=True)
  (dense3): Linear(in_features=15, out_features=2, bias=True)
)

In [16]:
env = gym.make('CartPole-v1')


In [17]:
import numpy as np

def sample_action(action_probs):
    p = action_probs
    p = p.ravel()
    '''
    cum_p= np.cumsum(p)
    
    r=np.random.random()
    
    for i in range(len(p)):
        
        if r<cum_p[i]:
            return i
        
    '''
    
    return np.random.choice(range(len(p)),p=p)
        
def discount(rewards):
    gamma =0.96
    
    rewards = np.array(rewards)
    discounted_rewards = np.zeros_like(rewards)
    
    for i in range(len(rewards)):
        cumsum=0
        for j in range(i,len(rewards)):
            cumsum+= gamma**(j-i) * rewards[j]
        
        discounted_rewards[i]=cumsum
        
    mean=discounted_rewards.mean()
    std=discounted_rewards.std()
    
    discounted_rewards-=mean
    discounted_rewards/=std
        
    return discounted_rewards
    
        

optimiser=torch.optim.Adam(net.parameters(),lr=0.01)
count=0
allrewards=[]
for i in range(1000):

    states,actions,rewards=[],[],[]
    state=env.reset()
    done=False
    steps=0

    while not done:
    
        states.append(state)
        state = torch.from_numpy(state).unsqueeze(0).float()
        action_probs = net(state).data.numpy().squeeze(0)
        

        action = sample_action(action_probs)
        actions.append(action)

        nextstate,reward,done,_ = env.step(action)
        rewards.append(reward)

        state = nextstate
        steps+=1
    
    states=np.array(states)
    actions=np.array(actions)
    rewards=np.array(rewards)
    
    states = torch.from_numpy(states).float()
    actions = torch.from_numpy(actions).long()
    
    action_prob_log = net(states)
    
    discounted_rewards = discount(rewards)
    discounted_rewards=torch.from_numpy(discounted_rewards).float()
    
    neg_logloss = F.cross_entropy(action_prob_log,actions,reduce=False)
    
    loss = torch.mean(neg_logloss * discounted_rewards)
    optimiser.zero_grad()
    
    loss.backward()
    
    optimiser.step()
    
    allrewards.append(rewards.sum())
    
    if rewards.sum()==500:
        count+=1
    else:
        count=0
        
    if count==6:
        print("Learnt")
        break
    
    if i%20==0:
        print(rewards.sum())
    
    
    
        
    
    
    

/Users/harshtiku/Desktop/exp/lib/python3.6/site-packages/torch/nn/functional.py:52: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


44.0
22.0
20.0
15.0
10.0
17.0
12.0
15.0
11.0
13.0
12.0
11.0
9.0
16.0
18.0
15.0
10.0
14.0
11.0
19.0
12.0
13.0
15.0
27.0
22.0
25.0
28.0
38.0
297.0
91.0
109.0
91.0
84.0
111.0
149.0
379.0
Learnt


In [18]:

env._max_episode_steps=1000
for i in range(10):
    cum=0
    state=env.reset()
    done=False
    while not done:
        state=torch.from_numpy(state).unsqueeze(0).float()
        action=net(state).data.numpy().argmax()
        
        #action=env.action_space.sample()

        nextstate,reward,done,_ = env.step(action)
        cum+=reward
        state=nextstate

        env.render()
    print(cum)
    env.close()

1000.0
1000.0
1000.0
1000.0
1000.0
1000.0


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/harshtiku/Desktop/exp/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-4a0e6d60d7bf>", line 9, in <module>
    action=net(state).data.numpy().argmax()
  File "/Users/harshtiku/Desktop/exp/lib/python3.6/site-packages/torch/nn/modules/module.py", line 477, in __call__
    result = self.forward(*input, **kwargs)
  File "<ipython-input-13-575048adb2c8>", line 16, in forward
    out = F.dropout(out)
  File "/Users/harshtiku/Desktop/exp/lib/python3.6/site-packages/torch/nn/functional.py", line 595, in dropout
    return _functions.dropout.Dropout.apply(input, p, training, inplace)
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/harshtiku/Desktop/exp/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1828, in showtraceback


KeyboardInterrupt: 

In [18]:
cum

158.0

In [14]:
count

10

In [28]:
allrewards

[21.0,
 14.0,
 11.0,
 13.0,
 20.0,
 17.0,
 20.0,
 15.0,
 10.0,
 22.0,
 11.0,
 18.0,
 36.0,
 14.0,
 13.0,
 11.0,
 10.0,
 29.0,
 25.0,
 30.0,
 32.0,
 11.0,
 13.0,
 15.0,
 19.0,
 30.0,
 14.0,
 13.0,
 11.0,
 10.0,
 23.0,
 50.0,
 18.0,
 13.0,
 10.0,
 24.0,
 30.0,
 13.0,
 27.0,
 17.0,
 19.0,
 16.0,
 16.0,
 23.0,
 16.0,
 21.0,
 43.0,
 17.0,
 18.0,
 26.0,
 23.0,
 22.0,
 29.0,
 16.0,
 13.0,
 13.0,
 31.0,
 14.0,
 10.0,
 13.0,
 40.0,
 15.0,
 15.0,
 22.0,
 23.0,
 17.0,
 15.0,
 13.0,
 10.0,
 21.0,
 20.0,
 18.0,
 14.0,
 22.0,
 16.0,
 30.0,
 23.0,
 26.0,
 20.0,
 29.0,
 50.0,
 19.0,
 11.0,
 13.0,
 20.0,
 10.0,
 17.0,
 30.0,
 20.0,
 16.0,
 16.0,
 19.0,
 13.0,
 24.0,
 26.0,
 10.0,
 12.0,
 21.0,
 9.0,
 10.0,
 10.0,
 35.0,
 10.0,
 17.0,
 24.0,
 12.0,
 11.0,
 15.0,
 23.0,
 34.0,
 14.0,
 28.0,
 47.0,
 16.0,
 13.0,
 29.0,
 16.0,
 21.0,
 14.0,
 17.0,
 10.0,
 14.0,
 10.0,
 12.0,
 13.0,
 17.0,
 25.0,
 26.0,
 21.0,
 42.0,
 16.0,
 21.0,
 21.0,
 22.0,
 12.0,
 15.0,
 14.0,
 34.0,
 17.0,
 22.0,
 26.0,
 15.0,
 15.0,


In [35]:
allrewards[-5:]

[65.0, 53.0, 59.0, 60.0, 67.0]

In [20]:
np.all(allrewards[-5:]==200)

False

In [3]:
import torch


In [6]:
d= torch.softmax(torch.from_numpy(np.random.random(5)),dim=-1).data.numpy()

In [10]:
d.ravel()

array([0.17043841, 0.20828897, 0.20552885, 0.27950469, 0.13623908])

In [11]:
np.random.choice(range(5),p=d)

0

In [20]:
np.array([env.observation_space.sample()])

array([[-7.3291397e-01,  9.9290386e+37, -5.2286819e-02,  2.6662688e+38]],
      dtype=float32)

In [22]:
200 == 200.0


True